## This notebook contains the metric evaluation for the test data

### Installing the specific version of tensorflow used for this project

In [ ]:
!pip install tensorflow==2.3.0

  Using cached tensorflow-2.3.0-cp37-cp37m-manylinux2010_x86_64.whl (320.4 MB)
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
ERROR: pip's depende

### Importing essential libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K 
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K

In [ ]:
print(tf.__version__)

2.3.0


In [ ]:
model_path = "/content/drive/MyDrive/Lightning/saved models"

### Defining the metrics used during the model building part

In [ ]:
def ETS(y_true, y_pred):
    ytrue = y_true
    ypred = K.sigmoid(y_pred)
    ypred = K.round(ypred)
    N1 = np.sum((ypred > 0) & (ytrue > 0))
    N2 = np.sum((ypred > 0) & (ytrue < 1))
    N3 = np.sum((ypred < 1) & (ytrue > 0))
    N4 = np.sum((ypred < 1) & (ytrue < 1))
    R = ((N1+N2)*(N1+N3))/(N1+N2+N3+N4)
    ets = (N1-R)/((N1+N2+N3)-R)
    return ets


def POD(y_true, y_pred):
    ytrue = y_true
    ypred = K.sigmoid(y_pred)
    ypred = K.round(ypred)
    true_positives = K.sum(ytrue * ypred)
    possible_positives = K.sum(ytrue)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def FAR(y_true, y_pred):
    ytrue = y_true
    ypred = K.sigmoid(y_pred)
    ypred = K.round(ypred)
    true_positives = K.sum(ytrue * ypred)
    predicted_positives = K.sum(ypred)
    precision = true_positives / (predicted_positives + K.epsilon())
    return 1 - precision

def weight_loss(y_true,y_pred):  # binary classification
    pw = 16
    ytrue = K.flatten(y_true)
    ypred = K.flatten(y_pred)
    return tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=ypred,labels=ytrue,pos_weight=pw))

def binary_acc(y_true,y_pred):
    ypred = K.sigmoid(y_pred)
    return K.mean(K.equal(y_true, K.round(ypred)), axis=-1)

### Loading the saved model

In [ ]:
model = tf.keras.models.load_model(model_path, custom_objects={'POD':POD, 'FAR':FAR, 'weight_loss':weight_loss, 'binary_acc':binary_acc})

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/core.py:1045: UserWarning: Lightning_Binary_Classifier is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


### Framing the test input.

In [ ]:
data_path = '/content/drive/MyDrive/Lightning/data/pre-processed data/BinaryLightningData.nc'
ds = xr.open_dataset(data_path).sel(time=slice("2014"))
channels = ["p80.162", "p79.162", "cape","r"]
step = 6

In [ ]:
dataset={}

for i in channels:
  if i == 'r':
    temp=ds[i].values[:,0,:,:]
  else:
    temp=ds[i].values
  scalers = {}
  for j in range(temp.shape[1]):
    scalers[j] = StandardScaler()
    temp[:, j, :] = scalers[j].fit_transform(temp[:, j, :]) 
  dataset[i]=temp
  print(i)

def preprocess_data(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps
		if end_ix > len(sequence)-1 or end_ix+6 > len(sequence)-1:
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:end_ix+6]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

def create_input(channels, ds_path, n_steps):
	stack = []
	for i in channels:
		input, _ = preprocess_data(dataset[i], n_steps=step)
		print(f"Adding channel {i} with shape: {input.shape}")
		stack.append(input)
	out = np.stack(stack, axis=-1)
	return out

test_input1 = create_input(channels, data_path, step)


#Encoder-2 Input 
test_input2,_ = preprocess_data(ds['Amplitude'].values, n_steps=step)
test_input2 = np.expand_dims(test_input2, axis=-1) 

test_input3 = test_input2[:,5,:,:]
test_input3 = np.expand_dims(test_input3, axis=1)

#Decoder Output
_, test_target = preprocess_data(ds['Amplitude'].values, n_steps=step)

test_target = np.expand_dims(test_target, axis=-1)
test_target = np.reshape(test_target, (-1,6, 29 * 33, 1))


test_input = [test_input1, test_input2, test_input3]

p80.162
p79.162
cape
r
Adding channel p80.162 with shape: (1452, 6, 29, 33)
Adding channel p79.162 with shape: (1452, 6, 29, 33)
Adding channel cape with shape: (1452, 6, 29, 33)
Adding channel r with shape: (1452, 6, 29, 33)


In [ ]:
test_predicted = model.predict(test_input, batch_size=8)
test_target = np.float32(test_target)

#print(test_predicted.shape)


test_predicted_1 = test_predicted[:,0,:,:]
np.expand_dims(test_predicted_1, axis=1)
test_predicted_2 = test_predicted[:,0:3,:,:]

test_target_1 = test_target[:,0,:,:]
np.expand_dims(test_target_1, axis=1)
test_target_2 = test_target[:,0:3,:,:]


print(test_target.shape)

(1452, 6, 957, 1)


### Calculating the metrics for the test data.

In [ ]:
#calculating the Six hour cumulative score 
pod_1 = POD(y_true=test_target, y_pred=test_predicted)
far_1 = FAR(y_true=test_target, y_pred=test_predicted)
ets_1 = ETS(y_true=test_target, y_pred=test_predicted)

#calculating the first hour cummulative score
pod_2 = POD(y_true=test_target_1, y_pred=test_predicted_1)
far_2 = FAR(y_true=test_target_1, y_pred=test_predicted_1)
ets_2 = ETS(y_true=test_target_1, y_pred=test_predicted_1)

#Calculating the first three hour cumulative score
pod_3 = POD(y_true=test_target_2, y_pred=test_predicted_2)
far_3 = FAR(y_true=test_target_2, y_pred=test_predicted_2)
ets_3 = ETS(y_true=test_target_2, y_pred=test_predicted_2)

In [ ]:
print('First hour score------------ POD: {pod}, FAR: {far}, ETS: {ets}'.format(pod=pod_2, far=far_2, ets=ets_2))
print('First three hour score------ POD: {pod}, FAR: {far}, ETS: {ets}'.format(pod=pod_3, far=far_3, ets=ets_3))
print('Six hour score-------------- POD: {pod}, FAR: {far}, ETS: {ets}'.format(pod=pod_1, far=far_1, ets=ets_1))

First hour score------------ POD: 0.5964369177818298, FAR: 0.446600079536438, ETS: 0.3854778526985041
First three hour score------ POD: 0.5049275159835815, FAR: 0.5527175664901733, ETS: 0.29203422682869873
Six hour score-------------- POD: 0.4369526505470276, FAR: 0.587054967880249, ETS: 0.25074688917854293
